# 앙상블 학습

In [8]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cancer_scaled = scaler.fit_transform(cancer.data)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_scaled, cancer.target, stratify=cancer.target,test_size=0.2, random_state=2021
)

# 앙상블 학습을 위한 분류기
- 로지스틱 회귀
- 서포트 벡터 머신
- K 최근접 이웃

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier

In [12]:
lrc = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [13]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LR', lrc), ('svc', svc), ('KNN', knn)], voting='hard'
)

In [14]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9824561403508771

- 개별 학습의 성능

In [15]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_train, y_train), svc.score(X_train, y_train), knn.score(X_train, y_train)

(0.967032967032967, 0.9824175824175824, 0.9692307692307692)

- 소프트 보팅

In [16]:
print(dir(lrc))

['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol', 'verbose', 'warm_start']


In [17]:
lrc.predict_proba(X_test[:5])

array([[0.36348222, 0.63651778],
       [0.97162943, 0.02837057],
       [0.1869565 , 0.8130435 ],
       [0.02133462, 0.97866538],
       [0.0548391 , 0.9451609 ]])

In [18]:
svc2 = SVC(probability=True)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:5])

array([[4.38993755e-01, 5.61006245e-01],
       [9.99798584e-01, 2.01416276e-04],
       [1.44150983e-02, 9.85584902e-01],
       [3.09675537e-03, 9.96903245e-01],
       [4.44845525e-03, 9.95551545e-01]])

In [19]:
voc2 = VotingClassifier(
    estimators=[('LR', lrc), ('SVC', svc2), ('KNN', knn)], voting='soft'
)
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test)

0.9912280701754386

In [20]:
voc2.predict_proba(X_test[:5])

array([[0.40516468, 0.59483532],
       [0.99048817, 0.00951183],
       [0.13372139, 0.86627861],
       [0.00810261, 0.99189739],
       [0.01971216, 0.98028784]])

- Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [23]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9736842105263158

#### XGBoost

In [24]:
import xgboost as xb
from xgboost import XGBClassifier

In [25]:
xgc = XGBClassifier()
xgc.fit(X_train, y_train)
xgc.score(X_test, y_test)

[12:04:56] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ICT01_08\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9912280701754386